In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions
from tensorflow.keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [3]:
import os
import shutil
import numpy as np
import random 
folder_path = r"dataset/animal_data/" # Put the path to your dataset
ratio_train = .8 # Ratio of images in the train dataset

directories = [name for name in os.listdir(folder_path) 
               if os.path.isdir(os.path.join(folder_path, name))]

directories.pop(-1)
directories.pop(-1)
print(directories)


for dir in directories: 
    print(dir)
    src_dir = folder_path+dir
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
    n_files = len(files)
    train_size = round(ratio_train * n_files)
    idx_train = random.sample(range(n_files), train_size)
    idx_test = list(set(range(n_files)) - set(idx_train))
    dst_dir = folder_path+'_train/'+dir+'/'

# Make sure destination folder exists
    os.makedirs(dst_dir, exist_ok=True)

# Loop over selected indices and copy files
    for idx in idx_train:
        file_name = files[idx]
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        shutil.copy(src_path, dst_path)
    dst_dir = folder_path+'_test/'+dir+'/'

# Make sure destination folder exists
    os.makedirs(dst_dir, exist_ok=True)

# Loop over selected indices and copy files
    for idx in idx_test:
        file_name = files[idx]
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        shutil.copy(src_path, dst_path)

['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']
Bear
Bird
Cat
Cow
Deer
Dog
Dolphin
Elephant
Giraffe
Horse
Kangaroo
Lion
Panda
Tiger
Zebra


dataloading


In [6]:
train_datagen=ImageDataGenerator(
    #rescale=1/255.0,
    rotation_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
train_generator=train_datagen.flow_from_directory(
    directory=r"dataset/animal_data/_train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=128,
    class_mode="categorical",
    shuffle=True,
    seed=42
)



test_datagen=ImageDataGenerator()
test_generator=test_datagen.flow_from_directory(
    directory=r"dataset/animal_data/_test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 1919 images belonging to 15 classes.
Found 905 images belonging to 15 classes.


CNN

In [8]:
model_cnn=Sequential([
    Input(shape=(224,224,3)),
    Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(15, activation='softmax')
])
model_cnn.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [11]:
model_cnn.fit(
    train_generator,
    epochs=5, validation_data=test_generator
)


Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 135s 9s/step - accuracy: 0.0615 - loss: 4.3708 - val_accuracy: 0.0729 - val_loss: 2.7082
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 122s 8s/step - accuracy: 0.0657 - loss: 2.7081 - val_accuracy: 0.0685 - val_loss: 2.7082
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 91s 6s/step - accuracy: 0.0677 - loss: 2.7081 - val_accuracy: 0.0685 - val_loss: 2.7082
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 87s 6s/step - accuracy: 0.0558 - loss: 2.7081 - val_accuracy: 0.0729 - val_loss: 2.7081
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 85s 6s/step - accuracy: 0.0688 - loss: 2.7081 - val_accuracy: 0.0729 - val_loss: 2.7081


VGG16

In [12]:
model=VGG16() #Import the VGG16 model

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


In [13]:
y_pred_vgg16=model.predict(test_generator)

905/905 ━━━━━━━━━━━━━━━━━━━━ 158s 175ms/step


In [14]:
predictions=decode_predictions(y_pred_vgg16)

first_predictions=list()
for pred in predictions:
    first_predictions.append(pred[0][1])

first_predictions=np.array(first_predictions)

35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


In [15]:
import collections
print("The three most predicted categories by VGG16 are:")
unique_pred=collections.Counter(first_predictions)
for animal in list(unique_pred.keys())[:3]:
    print(f"{animal}: {unique_pred[animal]}")

The three most predicted categories by VGG16 are:
ice_bear: 33
mushroom: 1
brown_bear: 17


Transfer learning VGG16

In [16]:
#Load pre-trained VGG-16 on ImageNet without the last fully-connected layers
model_2=VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))

# We do not train the layers in VGG16
for layer in model_2.layers:
    layer.trainable=False

In [17]:
model_3=Sequential([
    model_2,
    Flatten(),
    Dense(15, activation='softmax')
])



In [18]:
model_3.compile(
    loss="categorical_crossentropy",
    optimizer=SGD(learning_rate=0.0001, momentum=0.9),
    metrics=["accuracy"]
)

In [19]:
model_3.fit(
    train_generator,
    epochs=2,
    #verbose=2
)

Epoch 1/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 175s 12s/step - accuracy: 0.3715 - loss: 12.6176
Epoch 2/2
15/15 ━━━━━━━━━━━━━━━━━━━━ 176s 12s/step - accuracy: 0.7879 - loss: 3.0149


In [20]:
dic_scores["Augmented VGG16"]=model_3.evaluate(test_generator)[1]
print_results_dic(dic_scores)

905/905 ━━━━━━━━━━━━━━━━━━━━ 58109s 64s/step - accuracy: 0.9083 - loss: 1.1416


NameError: name 'dic_scores' is not defined